In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pickle
import pandas as pd
import plotly.io as pio

""" load data """
with open(os.path.join('..', 'dataset', 'experiments.pkl'), "rb") as experiments_file:
    experiments = pickle.load(experiments_file)

# display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 100000)
pio.renderers.default = "plotly_mimetype+notebook_connected"

# PID and MiCS Metal Oxide Semiconductor Sensor Timeseries

Below we show a small selection of timeseries recorded in our wind tunnel dataset.
As with the spatial reconstructions, we focus on one of the sampling experiments, where our 4 sensor probes traverse across the airstream periodically, while the entire gantry slowly steps forward in the direction of the wind. The four probes are mounted vertically above each other with an offset of 24cm, and also traverse up and down, such that we sample a grid of 12x12x8cm cells (see [spatial data notebook](spatial_data.ipynb)).

For some users, the timeseries themselves may be of interest, too. Out dataset contains the full, raw timeseries of all our recordings.
Additionally, we perform various preprocessing steps. For example, the raw voltages get converted to sensing element resistances in the case of metal-oxide semiconductor sensors. But we also compute ppm-equivalents of the target gasses, and subtract the background concentrations measured using two upstream sensors.

In the plots below, we show four interactive plots for the four sensors we used: MiCS 5524, MiCS 6814 (NO2 channel), MiCS 6814 (NH3 channel) and Alphasense PID-AH2.
for each plot, you can in your browser zoom into the timeseries as needed, as well as select whether you want to see the raw recorded voltage or the the background-subtracted ppm - the final stage of our preprocessing.

Do note that all signals are filtered with a rolling average filter of 2s, as some of the data is rather noisy, which makes it difficult to discern larger trends. If you require different filtering, you are free to clone this repository and run this notebook again yourself with changed parameters.


In [ ]:
import plotly.graph_objects as go

def plot2d(exp_idx, sensor_type, rolling_window=0, use_uniform_time=True):
    experiment = experiments["Sampling_Experiments"][exp_idx]

    fig = go.Figure()
    channels = [('upstream', i) for i in [0, 1]] + [('layers', i) for i in range(4)]
    title_suffix = f" for {sensor_type} in Experiment {exp_idx + 1}"
    titles = ["Voltages (raw)"+title_suffix, "ppm (background subtracted)"+title_suffix]
    for series_idx, series in enumerate(['voltage', 'ppm_relative']):
        for key, idx in channels:
            try:
                data = experiment[key][idx][sensor_type]
                data = data.rolling(f'{rolling_window}s').mean()

                fig.add_trace(go.Scattergl(
                    **(
                        {'x': data[series].index}                   # Sets the actual time steps from the DataFrame (more precise)
                        if not use_uniform_time else
                        {'x0': data[series].index[0], 'dx': 100}    # Set starting time + uniform time steps (smaller file sizes)
                    ),
                    y=data[series],
                    mode='lines',
                    name=f"{key} {idx}",
                    hovertemplate='Time: %{x}<br>Value: %{y:.2f}<extra></extra>',
                    visible=(series_idx==0),
                ))
            except KeyError:
                continue

    # Enable zooming, panning, range sliders, etc.
    fig.update_layout(
        title=titles[0],
        xaxis_title="Time",
        xaxis=dict(rangeslider=dict(visible=True), type='date'),
        template='plotly_white',
        height=500
    )

    # Add buttons to toggle visibility
    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                direction="right",
                buttons=list([
                    dict(label="voltage",
                         method="update",
                         args=[{"visible": 6*[True] + 4*[False]},
                               {"title": titles[0]},
                               {"yaxis":['y','y']},
                               ]),
                    dict(label="ppm",
                         method="update",
                         args=[{"visible": 6*[False] + 4*[True]},
                               {"title": titles[1]},
                               ]),
                ]),
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.1,
                xanchor="left",
                y=1.15,
                yanchor="top"
            ),
        ]
    )
    fig.show()

In [ ]:
plot2d(exp_idx=0, sensor_type='PID-sensor')

Note how the PID sensor signals for layers 0 and 1 spike much higher than layers 2 and 3. This shows that they traversed the plume much more directly than the others, which only peripherally hit it.
You can also see how in the beginning phase, there is no concentration visible (after subtracting the background). This is while the sensors are upstream of the source.
Once the sensor platforms reach the cells downstream of the source, we get strong readings.
As we increase the distance to the source, the readings magnitudes drop, but the signal starts to become spread out over time, which corresponds to a larger spatial extent of the plume at larger distances.

In [ ]:
plot2d(exp_idx=0, sensor_type='MiCS5524')

For all metal-oxide sensors, we see much longer recovery periods once the sensors have been exposed to gas.
This makes it a challenge to compute the true spatial concentrations, as this 'smearing out' effect needs to be compensated.
This is the topic of ongoing research in the gas sensing and electronic nose community.

This is why one could call the PID sensor in our experiments the 'gold standard', while the other sensors aim to evaluate the feasibility of using low-cost sensors in gas sensing applications.

In [ ]:
plot2d(exp_idx=0, sensor_type='MiCS6814_NO2')

Note here how the NO2 channel for the MiCS6814 is showing negative values for the ppm of NO2.
While this may initially seem surprising, it does make sense: The "NO2" channel of the MiCS6814 is sensitive to oxidizing gasses. Since the gas plume we generate in our experiments contains primarily reducing gasses, these act like "anti-oxidizing gasses" on the sensor surface – thus dropping the concentrations below zero.

In [ ]:
plot2d(exp_idx=0, sensor_type='MiCS6814_NH3')

The NH3 channel is again sensitive to reducing gasses, though has a different characteristic curve than the MiCCS5524 for example. It too exhibits the slow recovery as is visible in the other metal-oxide sensor timeseries, but is otherwise also sensitive to the constituents of the fog plume released in the wind tunnel experiments.